In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Default = 'https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_DefaultName = 'Deliberate_v2.safetensors' #@param {type:"string"}
#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

#@markdown -----
#@markdown *실행속도*

#@markdown **ControlNet을 사용하지 않으면 체크를 해제해주세요**
ControlNet = True #@param {type:"boolean"}

#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = True #@param {type:"boolean"}

#@markdown **After Detailer(adetailer)를 사용하지 않으면 체크를 해제해주세요**
ADetailer = False #@param {type:"boolean"}

#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False #@param {type:"boolean"}

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/output
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lora
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/embedding
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/wildcards

checkpoint_url = Checkpoint_Default
checkpoint_name = Checkpoint_DefaultName
checkpoint_down_path = '/content/sdd-webui/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  checkpoint_down_path = '/content/sdd-webui/models/Stable-diffusion/google'

checkpoint_file = checkpoint_down_path+'/'+checkpoint_name

# https://github.com/camenduru 참고

%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2 libcairo2-dev
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

if DDetailer:
  !pip install -U openmim
  !mim install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html
  !pip install mmdet==2.28.2

!git clone -b stable1-controlnet https://github.com/neuralninja22/sd-webui /content/sdd-webui
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/sdd-webui/models/ESRGAN -o 4x-UltraSharp.pth
if DDetailer:
  !git clone https://github.com/ninjaneural/ddetailer /content/sdd-webui/extensions/ddetailer
if ADetailer:
  !git clone https://github.com/Bing-su/adetailer /content/sdd-webui/extensions/adetailer
if SegmentAnything:
  !git clone https://github.com/ninjaneural/segment-anything /content/sdd-webui/extensions/segment-anything
!git clone https://github.com/adieyal/sd-dynamic-prompts /content/sdd-webui/extensions/sd-dynamic-prompts
!git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding /content/sdd-webui/extensions/sd-dynamic-thresholding
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/sdd-webui/extensions/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/sdd-webui/extensions/ultimate-upscale-for-automatic1111
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/sdd-webui/extensions/multidiffusion-upscaler-for-automatic1111

!git clone -b stable1 https://github.com/ninjaneural/deforum /content/sdd-webui/extensions/deforum
!git clone -b stable1 https://github.com/neuralninja22/images-browser /content/sdd-webui/extensions/images-browser
!git clone -b stable https://github.com/ninjaneural/additional-networks /content/sdd-webui/extensions/additional-networks
!git clone https://github.com/neuralninja22/wd14-tagger /content/sdd-webui/extensions/wd14-tagger
!git clone https://github.com/ninjaneural/photopea-embed /content/sdd-webui/extensions/photopea-embed
!git clone https://github.com/ninjaneural/video-util /content/sdd-webui/extensions/video-util

!git clone https://github.com/fkunn1326/openpose-editor /content/sdd-webui/extensions/openpose-editor
!git clone https://github.com/hnmr293/posex /content/sdd-webui/extensions/posex
!git clone https://github.com/fishslot/video_loopback_for_webui /content/sdd-webui/extensions/video_loopback_for_webui

%cd /content/sdd-webui
!git reset --hard

!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/config.json -O /content/sdd-webui/config.json

# 구글드라이브 연결
if Google_Drive:
  !mkdir /content/sdd-webui/models/Lora
  !mkdir /content/sdd-webui/models/hypernetworks
  !rm /content/sdd-webui/embeddings/google
  !rm /content/sdd-webui/models/Stable-diffusion/google
  !rm /content/sdd-webui/models/Lora/google
  !rm /content/sdd-webui/models/hypernetworks/google
  !rm /content/sdd-webui/outputs
  !mv /content/sdd-webui/outputs /content/sdd-webui/outputs_backup
  !rm /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards
  !mv /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards_backup
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/embedding /content/sdd-webui/embeddings/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint /content/sdd-webui/models/Stable-diffusion/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/lora /content/sdd-webui/models/Lora/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork /content/sdd-webui/models/hypernetworks/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/output /content/sdd-webui/outputs
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/wildcards /content/sdd-webui/extensions/sd-dynamic-prompts/wildcards
else:
  !rm /content/sdd-webui/outputs

# controlnet models
if ControlNet:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d /content/sdd-webui/extensions/controlnet/models -o diff_control_sd15_temporalnet_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d /content/sdd-webui/extensions/controlnet/models -o t2iadapter_style_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d /content/sdd-webui/extensions/controlnet/models -o t2iadapter_color_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d /content/sdd-webui/extensions/controlnet/models -o t2iadapter_sketch_sd15v2.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d /content/sdd-webui/extensions/controlnet/models -o t2iadapter_zoedepth_sd15v1.pth

# sam 
if SegmentAnything:
  !mkdir /content/sdd-webui/models/sam
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -d /content/sdd-webui/models/sam -o sam_vit_h_4b8939.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{checkpoint_url}" -d {checkpoint_down_path} -o {checkpoint_name}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {checkpoint_down_path} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/sdd-webui/models/VAE -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/sdd-webui/models/Lora -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/63006?type=Model&format=SafeTensor" -d /content/sdd-webui/models/Lora -o LowRA.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16" -d /content/sdd-webui/models/Lora -o blindbox_V1Mix.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/sdd-webui/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/sdd-webui/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d /content/sdd-webui/embeddings -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/sdd-webui/embeddings/ -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/sdd-webui/embeddings/ -o bad_prompt_version2.pt

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/sdd-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/sdd-webui/launch.py

if Ngrok_Key:
  !python launch.py --xformers --no-half-vae --theme dark --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue --cloudflared
